# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import datetime
import time
import dateutil.parser
import requests
from typing import List, Dict

%matplotlib inline
plt.rcParams['figure.figsize'] = [17, 7]

# Get Order Book Data

In [25]:
def orderbook_data(depth: int, contract: str = "ETHUSD", raw_data=False):

    url = "https://www.bitmex.com/api/v1/orderBook/L2"
    book_data = {'buys': 0, 'sells': 0}
    raw_book_data = {'total_buys': 0, 'sum_of_buys': 0, 'furthest_buy_price': 0,
                    'total_sells': 0, 'sum_of_sells': 0, 'furthest_sell_price': 0}

    # Set inputs
    data = dict()
    data['symbol'] = contract
    data['depth'] = depth

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        response_data = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if response_data is not None:
        if raw_data:
            for idx, data in enumerate(response_data):

                if idx == 0:
                    raw_book_data['furthest_sell_price'] = data['price']

                if idx == len(response_data)-1:
                    raw_book_data['furthest_buy_price'] = data['price']

                if data['side'] == "Buy":
                    raw_book_data["sum_of_buys"] += data["size"]
                    raw_book_data['total_buys'] += 1

                elif data['side'] == "Sell":
                    raw_book_data["sum_of_sells"] += data["size"]
                    raw_book_data['total_sells'] += 1

            return raw_book_data

        else:
            for data in response_data:
                if data['side'] == "Buy":
                    book_data["buys"] += data["size"]

                elif data['side'] == "Sell":
                    book_data["sells"] += data["size"]
    
            return book_data

# Get current price

In [23]:
def get_curr_price(contract: str) -> str:

    url = "https://www.bitmex.com/api/v1/trade/bucketed"

    data = dict()
    data['symbol'] = contract
    data['partial'] = True  # returns a candle if it is not finished yet
    data['binSize'] = '1m'
    data['count'] = 1   # how many candles we can return (500 max)
    data['reverse'] = True

    try:
        response = requests.get(url, params=data)
    except Exception as e:
        print(f"Connecton error while making GET request to {url}: {e}")
        return

    if response.status_code == 200:
        raw_candles = response.json()
    else:
        print(f"Error while making GET request to {url}: {response.status_code}")
        print(response.headers)
        return None
    
    if raw_candles is not None:
        return raw_candles[0]['close']

# Get Size of the OrderBook

In [27]:
contract = "XBTUSD"

book_size = orderbook_data(0, contract, raw_data=True)
book_size_avg = (book_size["total_buys"] + book_size["total_sells"]) // 2

curr_price = get_curr_price(contract)


print(f"\nThe average size of the order book for {contract} is {book_size_avg}")
print(f'\nTotal buys: {book_size["total_buys"]}')
print(f'Total sells: {book_size["total_sells"]}')
print(f"\nCurrent price ${curr_price}")
print(f'Furthest buy price: ${book_size["furthest_buy_price"]}')
print(f'Furthest sell price: ${book_size["furthest_sell_price"]}')


The average size of the order book for XBTUSD is 2643

Total buys: 1919
Total sells: 3367

Current price $19168
Furthest buy price: $0.5
Furthest sell price: $1000000


In [11]:
depth = 500
contract = "XBTUSD"

book_data = orderbook_data(depth, contract, raw_data=False)

ret = "Buyers" if book_data["buys"] > book_data["sells"] else "sellers"

if ret == "Buyers":
    amount = book_data["buys"] - book_data["sells"]
else:
    amount = book_data["sells"] - book_data["buys"]


print(book_data)
print(f"\nThere are {amount} more order values from {ret}")

TypeError: list indices must be integers or slices, not str